<a href="https://colab.research.google.com/github/erikapaceep/NLP/blob/main/MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLM Training Logic 

In [1]:
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

In [3]:
#initialize the tokenizer and the model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
text = ("After Abraham Lincoln won the November 1860 presidential [MASK] on an "
        "anti-slavery platform, an initial seven slave states declared their "
        "secession from the country to form the Confederacy. War broke out in "
        "April 1861 when secessionist forces [MASK] Fort Sumter in South "
        "Carolina, just over a month after Lincoln's inauguration.")

In [5]:
inputs = tokenizer(text, return_tensors='pt')

In [6]:
outputs = model(**inputs)

In [7]:
# we are just getting the prediction logit
outputs.keys()

odict_keys(['logits'])

In [8]:
outputs.logits.shape

torch.Size([1, 62, 30522])

In [9]:
# in reality we need to mask tokens randomly which is not what we have done and we also need an unmasked vesion of our input as label to feed the model peer and calcualte our loss

In [10]:
text = ("After Abraham Lincoln won the November 1860 presidential election on an "
        "anti-slavery platform, an initial seven slave states declared their "
        "secession from the country to form the Confederacy. War broke out in "
        "April 1861 when secessionist forces attacked Fort Sumter in South "
        "Carolina, just over a month after Lincoln's inauguration.")

In [11]:
# and again we need to convert this into our tokenized tensor
inputs = tokenizer(text, return_tensors='pt')

In [12]:
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,  2602,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,  2749,  4457,  3481,  7680,  3334,  1999,  2148,  3792,
          1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [13]:
# in order to mask we care about the input ids tensors and what we want is to mask around 15% of the input ids
# rather than than masking 15% of the token we want to have each single token to be masked with 15% of probs of being mask
rand = torch.rand(inputs.input_ids.shape)
rand
# we have a random value form 0 to 1 so we can create a masking rate base if those are

tensor([[0.5783, 0.3194, 0.6915, 0.6029, 0.0651, 0.8332, 0.7541, 0.2314, 0.6311,
         0.2503, 0.6493, 0.7478, 0.9389, 0.1346, 0.5387, 0.6014, 0.3046, 0.5034,
         0.7244, 0.3087, 0.9287, 0.2992, 0.5226, 0.2544, 0.7987, 0.9322, 0.5210,
         0.8929, 0.6981, 0.9389, 0.7229, 0.2063, 0.8215, 0.9797, 0.8878, 0.6280,
         0.2964, 0.2415, 0.5708, 0.4243, 0.1869, 0.2067, 0.4400, 0.9225, 0.2711,
         0.9365, 0.9681, 0.6603, 0.2664, 0.6972, 0.9720, 0.1237, 0.3556, 0.0311,
         0.3738, 0.2408, 0.2354, 0.2000, 0.9306, 0.5210, 0.2216, 0.9437]])

In [14]:
inputs.input_ids != 102

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False]])

In [15]:
mask_arr = (rand < 0.15) & (inputs.input_ids != 102)
mask_arr = (rand < 0.15) * (inputs.input_ids != 102)
mask_arr

tensor([[False, False, False, False,  True, False, False, False, False, False,
         False, False, False,  True, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False,  True, False,  True, False, False, False, False, False, False,
         False, False]])

In [16]:
#index where we have prob < 0.15 
mask_arr[0].nonzero()

tensor([[ 4],
        [13],
        [51],
        [53]])

In [17]:
# to get this into a list
selection = torch.flatten(mask_arr[0].nonzero()).tolist()
selection

[4, 13, 51, 53]

In [18]:
# copy the original tensor into a new one called label
inputs['labels']=inputs.input_ids.detach().clone()
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,  2602,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,  2749,  4457,  3481,  7680,  3334,  1999,  2148,  3792,
          1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([

In [19]:
# now we can mask our input ids 
inputs.input_ids[0, selection] = 103

In [20]:
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,   103,  1996,  2281,  7313,  4883,  2602,
          2006,  2019,  3424,   103,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,  2749,  4457,  3481,  7680,  3334,  1999,  2148,  3792,
          1010,   103,  2058,   103,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([

In [21]:
outputs = model(**inputs)

In [22]:
outputs.keys()

odict_keys(['loss', 'logits'])

Now we have loss on the top of it, because we have also added labels:

In [23]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [24]:
outputs.loss

tensor(0.5380, grad_fn=<NllLossBackward0>)

In [25]:
import requests

In [26]:
data = requests.get('https://raw.githubusercontent.com/jamescalam/transformers/main/data/text/meditations/clean.txt')

In [27]:
text = data.text

In [28]:
#using text to fine tune our model

In [29]:
# 1. tokenize out model
# we need to split by new line of character, which will give us a list rather than a single string here
inputs = tokenizer(text.split('\n'), return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [30]:
inputs.input_ids.shape

torch.Size([507, 512])

In [31]:
inputs.input_ids.dtype

torch.int64

In [32]:
# we need to create our label tensor by cloning the input_id
inputs['labels'] = inputs.input_ids.detach().clone()

In [33]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [34]:
# now we need to mask our input ids: create a random tensor
rand = torch.rand(inputs.input_ids.shape)
# this time we add another condition as we don't want to mask our padding token either (which is our padding token)
rand_mask = (rand < 0.15) & (inputs.input_ids != 102) & (inputs.input_ids != 101) & (inputs.input_ids != 0)
rand_mask

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [35]:
rand_mask.shape[0]

507

In [36]:
rand_mask[0].nonzero()

tensor([[ 7],
        [15]])

In [37]:
torch.flatten(rand_mask[0].nonzero()).tolist()

[7, 15]

In [38]:
# since now we have multiple list we need a single array for each one of those vectors
# we initialize our selection as a list and then rather then loop through all the vectors here
seleciton = []

for i in range(inputs.input_ids.shape[0]):
  sel = torch.flatten(rand_mask[i].nonzero()).tolist()
  selection.append(sel)

In [39]:
for i in range(inputs.input_ids.shape[0]):
  inputs.input_ids[i, selection[i]] = 103

In [40]:
inputs.input_ids

tensor([[  101,  2013,  2026,  ...,     0,     0,     0],
        [  101,  2013,  1996,  ...,     0,     0,     0],
        [  101,  2013,  2026,  ...,     0,     0,     0],
        ...,
        [  101,  3459,  2185,  ...,     0,     0,     0],
        [  101,  2043, 15223,  ...,     0,     0,     0],
        [  101,  7887,  3288,  ...,     0,     0,     0]])

In [41]:
inputs

{'input_ids': tensor([[  101,  2013,  2026,  ...,     0,     0,     0],
        [  101,  2013,  1996,  ...,     0,     0,     0],
        [  101,  2013,  2026,  ...,     0,     0,     0],
        ...,
        [  101,  3459,  2185,  ...,     0,     0,     0],
        [  101,  2043, 15223,  ...,     0,     0,     0],
        [  101,  7887,  3288,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[  101,  2013,  2026,  ...,     0,     0,     0],
        [  101,  2013,  1996,  ...,     0,     0,     0],
        [  101,  2013, 

--------------------------------------------------------------------------------

In [42]:
# To feed the model during training we need to use the so called data loader and a dataloder require we to feed a pythorch data object based on our meditation dataset
# this class will be set up as a dataset object,
class MeditationsDataset(torch.utils.data.Dataset):
  # we need to define a initialization object here
  def __init__(self,encodings):
    self.encodings = encodings
  
  # this is going to retunrn the number of sample we have in our dataset (i.e. 507)
  def __len__(self):
    return len(self.encodings.input_ids)
  
  #we need to define the get item method
  def __getitem__(self, idx):
    # return a dictionary which contains the input ids
    # this is to get a single tensor for this set
    # Given a certain index returning the tensor of that index
    return {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}

In [43]:
# initialize the data using this class
dataset = MeditationsDataset(inputs)

In [44]:
type(inputs)

transformers.tokenization_utils_base.BatchEncoding

In [45]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [46]:
# initialize the dataloader
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [47]:
# setting up the training loop 
#1. set up GPU and CPU usage

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [48]:
device

device(type='cuda')

In [49]:
# we moved our model over a GPU or a CPU
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [50]:
#activate the training mode of our model
model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [51]:
#we are going to be using an optimizer form transformers
from transformers import AdamW # reduce overfitting - usually very easy to overfit

#initialize our optimizer
optim = AdamW(model.parameters(),lr=5e-5)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# wrap everything using the tqdm, to see the progress of training
from tqdm import tqdm

for epoch in range(2):
  # we are going to loop through each of the batches provided by our data loader
  loop = tqdm(loader, leave=True)
  # then we loop though each batch
  for batch in loop:
    optim.zero_grad() # this stops form carry the calculations form the previous step into the next
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    outputs = model(input_ids, attention_mask = attention_mask, labels = labels)

    loss = outputs.loss
    loss.backward() # the loss for every param
    optim.step() # this is to update the param

    loop.set_description(f'Epoch {epoch}')
    loop.set_postfix(loss=loss.item())

We learn how to train a model using MLM in pythorc using huggigface transformers using something we call a trainer.
We initialize a training argument object so we import that from transformers

In [54]:
from transformers import TrainingArguments
# define the training arguments
arg = TrainingArguments(
    output_dir='out',
    # specify the batch size
    per_device_train_batch_size=16,
    num_train_epochs=2
)

In [56]:
#now we need to initialize our trianer
from transformers import Trainer

trainer  =  Trainer(
    model=model,
    args=arg,
    train_dataset=dataset
)

In [ ]:
# our model will begin training when we are going to execute this
trainer.train()